In [ ]:
import numpy as np
import time
from pathlib import Path
from pydrake.all import (
    BasicVector,
    ConstantVectorSource,
    Context,
    DiagramBuilder,
    LeafSystem,
    MultibodyPlant,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Trajectory, 
    PiecewisePolynomial,
    PiecewisePose,
    JacobianWrtVariable,
    StartMeshcat,
    TrajectorySource,
    Integrator,
    InverseKinematics,
    Solve,
    
)

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)
from manipulation.exercises.trajectories.rrt_planner.robot import (
    ConfigurationSpace,
    Range,
)
from manipulation.exercises.trajectories.rrt_planner.rrt_planning import (
    RRT,
    Problem,
    TreeNode,
)
import h5py
import os
import random
from random import random as rand_fn
from typing import Literal

In [ ]:
meshcat = StartMeshcat()


INFO:drake:Meshcat listening for connections at http://localhost:7000


In [ ]:
DRAWER_CONFIGS = {
    "label53": {"handle_label": "handle5", "r_min": 0.3, "r_max": 0.335, "lambda": 0.00004, "kp_null": 1},
    "label62": {"handle_label": "handle3", "r_min": 0.4, "r_max": 0.45, "lambda": 0.004, "kp_null": 5},
    "label31": {"handle_label": "handle3", "r_min": 0.39, "r_max": 0.43,"lambda": 0.0004, "kp_null": 1.5}
}


In [ ]:
def get_random_drawer_pose(
    drawer_name,  
    fov_deg=90, 
    face_noise_deg=30,
    cabinet_depth=1,
    cabinet_width=1,    
):
    """
    Returns (x, y, yaw_degrees)
    """
    r_min, r_max = DRAWER_CONFIGS[drawer_name]["r_min"], DRAWER_CONFIGS[drawer_name]["r_max"]
    r = random.uniform(r_min, r_max)
    print(r)
    theta_rad = np.radians(random.uniform(270-fov_deg/2, 270+fov_deg/2))
    
    x = r * np.cos(theta_rad)
    y = r * np.sin(theta_rad)
    
    
    # Adjust angle b/w drawer origin is bottom right corner of cabinet
    perfect_yaw_rad = np.arctan2(-y, -x)
    yaw_rad = perfect_yaw_rad + np.radians(random.uniform(-face_noise_deg, face_noise_deg))
    
    local_center_x = cabinet_depth / 2.0
    local_center_y = cabinet_width / 2.0
    
    c_yaw = np.cos(yaw_rad)
    s_yaw = np.sin(yaw_rad)
    
    world_offset_x = local_center_x * c_yaw - local_center_y * s_yaw
    world_offset_y = local_center_x * s_yaw + local_center_y * c_yaw
    
    final_origin_x = x - world_offset_x
    final_origin_y = y - world_offset_y
    
    return final_origin_x, final_origin_y, np.degrees(yaw_rad)
    


In [ ]:
def generate_scenario_string(drawer_name = "label53", **kwargs) -> str:
    drawer_urdf_path = f"{Path.cwd()}/urdf/custom/output/{drawer_name}.urdf"
    
    x, y, yaw = get_random_drawer_pose(drawer_name = drawer_name, **kwargs)
    
    scenario_string = f"""
    directives:
    # add robot
    - add_model:
        name: iiwa
        file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
        default_joint_positions:
            iiwa_joint_1: [-1.57]
            iiwa_joint_2: [0.1]
            iiwa_joint_3: [0]
            iiwa_joint_4: [-1.2]
            iiwa_joint_5: [0]
            iiwa_joint_6: [1.6]
            iiwa_joint_7: [0]
    - add_weld:
        parent: world
        child: iiwa::iiwa_link_0

    # add gripper
    - add_model:
        name: wsg
        file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
    - add_weld:
        parent: iiwa::iiwa_link_7
        child: wsg::body
        X_PC:
            translation: [0, 0, 0.09]
            rotation: !Rpy {{ deg: [90, 0, 90]}}

    # add camera mounted to world 
    - add_frame:
        name: camera0_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [250, 0, 180.0]}}
            translation: [0, 2, 1.4]
    - add_model:
        name: camera0
        file: package://manipulation/camera_box.sdf
    - add_weld:
        parent: camera0_origin
        child: camera0::base

    # add camera mounted to robot wrist
    - add_frame:
        name: camera_wrist
        X_PF:
            base_frame: iiwa::iiwa_link_7
            translation: [-0.05, 0, 0.07]   
            rotation: !Rpy {{deg: [0, 0, -90]}}
    - add_model:
        name: camera1
        file: package://manipulation/camera_box.sdf
    - add_weld:
        parent: iiwa::camera_wrist
        child: camera1::base

    # add camera mounted to world pointing top down
    - add_frame:
        name: camera2_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [0, 180.0, 0.0]}}
            translation: [0, -0.5, 3.0]
    - add_model:
        name: camera2
        file: package://manipulation/camera_box.sdf
    - add_weld:
        parent: camera2_origin
        child: camera2::base
        
    # add random drawer
    - add_model:
        name: drawer
        file: file://{drawer_urdf_path}
    - add_frame:
        name: drawer_origin
        X_PF:
            base_frame: world
            translation: [{x}, {y}, 0]    
            rotation: !Rpy {{ deg: [0, 0, {yaw}]}}  
    - add_weld:
        parent: drawer_origin
        child: drawer::base_link
        
    cameras:
        camera0:
            name: camera0
            depth: True
            X_PB:
                base_frame: camera0::base
        camera1:
            name: camera1
            depth: True
            focal: !FovDegrees
                x: 110   # horizontal FOV in degrees
            X_PB:
                base_frame: camera1::base
        camera2:
            name: camera2
            depth: True
            X_PB:
                base_frame: camera2::base
    
    model_drivers:
        iiwa: !IiwaDriver
            control_mode: position_only 
            hand_model_name: wsg
        wsg: !SchunkWsgDriver {{}}
    """
    return scenario_string

scenario_string = generate_scenario_string(drawer_name = "label31")
scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder = DiagramBuilder()
builder.AddSystem(station)
diagram = builder.Build()
context = diagram.CreateDefaultContext()

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.Initialize()

0.4178583526143932


material [ 'None_NONE.006' ] not found in .mtl



In [ ]:
def grasp_pose(X_WO: RigidTransform) -> RigidTransform:
    p_OG = np.array([0.122, 0.01, 0.03])

    R_OG = RotationMatrix.MakeZRotation(np.pi/2).multiply(RotationMatrix.MakeYRotation(np.pi))

    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO @ X_OG
    return X_WG

def approach_pose(X_WG: RigidTransform) -> RigidTransform:
    R_WG = X_WG.rotation()
    p_WG = X_WG.translation()

    # Get the approach direction: negative X-axis of gripper frame (backward from face)
    # This is the first column of the rotation matrix, negated
    approach_direction = -R_WG.col(1)

    # Move 20cm (0.2m) in front of the cabinet and 5cm down
    p_WGApproach = p_WG + 0.12 * approach_direction  # + np.array([0, 0, 0.05])

    X_WGApproach = RigidTransform(R_WG, p_WGApproach)
    return X_WGApproach

def goal_pose(X_W_Grasporiginal: RigidTransform, X_W_Handle: RigidTransform) -> RigidTransform:
    v_obj_move = np.array([0.3, 0, 0])
    v_world_move = X_W_Handle.rotation().multiply(v_obj_move)

    p_W_Grasp = X_W_Grasporiginal.translation()
    p_W_Goal = p_W_Grasp + v_world_move

    X_W_Graspgoal = RigidTransform(X_W_Grasporiginal.rotation(), p_W_Goal) 
    return X_W_Graspgoal 

def post_goal_pose(X_WGgoal: RigidTransform, X_W_Handle: RigidTransform) -> RigidTransform:
    v_obj_move = np.array([0.23, 0, 0])

    v_world_move = X_W_Handle.rotation().multiply(v_obj_move)

    p_W_Ggoal = X_WGgoal.translation()
    p_W_Gpostgoal = p_W_Ggoal + v_world_move

    X_W_Gpostgoal = RigidTransform(X_WGgoal.rotation(), p_W_Gpostgoal)
    return X_W_Gpostgoal

In [ ]:
# def visualize_grasp_poses(X_WO: RigidTransform, meshcat_instance, handle_name: str = "handle"):
#     """Visualize the grasp, pregrasp, goal, and post-goal poses using AddMeshcatTriad."""
#     from manipulation.meshcat_utils import AddMeshcatTriad

#     # Compute all poses
#     X_WG = grasp_pose(X_WO)
#     X_WG_approach = approach_pose(X_WG)
#     X_WG_goal = goal_pose(X_WG, X_WO)
#     X_WG_postgoal = post_goal_pose(X_WG_goal, X_WO)

#     # # Draw coordinate frames using AddMeshcatTriad
#     # AddMeshcatTriad(meshcat_instance, "grasp_pose", X_PT=X_WG, length=0.05, radius=0.003, opacity=0.7)
#     # AddMeshcatTriad(meshcat_instance, "pregrasp_pose", X_PT=X_WG_approach, length=0.05, radius=0.003, opacity=0.7)
#     # AddMeshcatTriad(meshcat_instance, "goal_pose", X_PT=X_WG_goal, length=0.05, radius=0.003, opacity=0.7)
#     # AddMeshcatTriad(meshcat_instance, "postgoal_pose", X_PT=X_WG_postgoal, length=0.05, radius=0.003, opacity=0.7)

#     print("Poses visualized:")
#     print(f"  Grasp position: {X_WG.translation()}")
#     print(f"  Pregrasp position: {X_WG_approach.translation()}")
#     print(f"  Goal position: {X_WG_goal.translation()}")
#     print(f"  Post-goal position: {X_WG_postgoal.translation()}")

In [ ]:
# Example: Visualize pregrasp poses
# Get the current drawer handle pose from the simulation
plant = station.GetSubsystemByName("plant")
plant_context = plant.GetMyContextFromRoot(context)
drawer_model_instance = plant.GetModelInstanceByName("drawer")

handle_name = "handle3"  # Change to your drawer's handle
handle_body = plant.GetBodyByName(handle_name, drawer_model_instance)
X_W_Handle = plant.EvalBodyPoseInWorld(plant_context, handle_body)

# # Visualize the poses
# visualize_grasp_poses(X_W_Handle, meshcat)

In [ ]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant: MultibodyPlant, drawer_name = "label53"):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame() 
        self._W = plant.world_frame()

        # Declaring the ports
        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()
        self.drawer_name =drawer_name

    def CalcOutput(self, context: Context, output: BasicVector):
        V_WG_desired = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
    
        # Using the jacobian to determine the desired joint velocity
        J_G_W = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._G, 
            [0, 0, 0], 
            self._W, 
            self._W,
        )
        
        # Extract the values corresponding to the IIWA joints
        J_iiwa = J_G_W[:, self.iiwa_start : self.iiwa_end + 1]

        # Damp least squares to fix near singularity issues
        # higher lambda -> more staable but less accurate
        # lambda_val = 0.00004
        lambda_val = DRAWER_CONFIGS[self.drawer_name]["lambda"]
        
        # Formula: v = J.T * inv(J * J.T + lambda^2 * I) * V_desired
        J_T = J_iiwa.T
        n_rows = J_iiwa.shape[0] # 6
        
        # Calculate (J * J.T + lambda^2 * I)
        M = J_iiwa @ J_T + (lambda_val**2) * np.eye(n_rows)
        
        v_task = J_T @ np.linalg.solve(M, V_WG_desired)


        # Null space control to fix near joint limitations
        
        # values close to limits of joints
        q_nominal = np.array([0, 0.6, 0, -1.2, 0, 1.6, 0]) 
        
        # kp_null = 1
        kp_null = DRAWER_CONFIGS[self.drawer_name]["kp_null"]
        v_secondary_desired = kp_null * (q_nominal - q)
        
        J_pinv = np.linalg.pinv(J_iiwa) 
        n_joints = J_iiwa.shape[1] # 7
        P = np.eye(n_joints) - (J_pinv @ J_iiwa)
        
        v_null = P @ v_secondary_desired

        # Combine
        v_total = v_task + v_null
        
        v_clamped = np.clip(v_total, -2.0, 2.0)

        output.SetFromVector(v_clamped)

In [ ]:
WSG_MIN = 0.0        # fully closed width (meters)
WSG_MAX = 0.1        # fully open width (meters)

def normalize_wsg(width):
    return np.clip((width - WSG_MIN) / (WSG_MAX - WSG_MIN), 0.0, 1.0)

def unnormalize_wsg(norm_width):
    return WSG_MIN + norm_width * (WSG_MAX - WSG_MIN)

# TODO: figure out real velocity limit in simulation by closing/opening grippers
WSG_MAX_SPEED = 0.4  # meters/sec width change

def normalize_wsg_velocity(width_velocity):
    v = width_velocity / WSG_MAX_SPEED
    return np.clip(v, -1.0, 1.0)

In [ ]:
class DrawerProblem(Problem):
    """Problem class for RRT-Connect planning in drawer opening configuration space."""
    def __init__(
        self,
        q_start: np.ndarray,
        q_goal: np.ndarray,
        plant: MultibodyPlant,
        scene_graph,
        station,
        station_context,
        drawer_name: str = "label53",
    ) -> None:
        self.drawer_name = drawer_name
        self.plant = plant
        self.plant_context = plant.CreateDefaultContext()
        self.iiwa_model = plant.GetModelInstanceByName("iiwa")
        self.scene_graph = scene_graph
        self.station = station
        self.station_context = station_context
        self.query_output_port = scene_graph.GetOutputPort("query") if scene_graph is not None else None
        
        # Construct configuration space for IIWA
        nq = 7
        joint_limits = np.zeros((nq, 2))
        for i in range(nq):
            joint = plant.GetJointByName(f"iiwa_joint_{i + 1}")
            joint_limits[i, 0] = joint.position_lower_limits().item()
            joint_limits[i, 1] = joint.position_upper_limits().item()

        range_list = [Range(joint_limit[0], joint_limit[1]) for joint_limit in joint_limits]

        def l2_distance(q: tuple):
            return np.sqrt(sum(q_i**2 for q_i in q))

        max_steps = nq * [np.pi / 180]  # two degrees
        cspace_iiwa = ConfigurationSpace(range_list, l2_distance, max_steps)

        # Call base class constructor
        Problem.__init__(
            self,
            x=10,
            y=10,
            robot=None,
            obstacles=None,
            start=tuple(q_start),
            goal=tuple(q_goal),
            cspace=cspace_iiwa,
        )

    def collide(self, configuration: np.ndarray) -> bool:
        """Check if configuration is in collision."""
        q = np.array(configuration)
        self.plant.SetPositions(self.plant_context, self.iiwa_model, q)
        if self.query_output_port is not None and self.scene_graph is not None:
            # Get the correct context from the station, not the plant
            scene_graph_context = self.scene_graph.GetMyContextFromRoot(self.station_context)
            query_object = self.query_output_port.Eval(scene_graph_context)
            collision_pairs = query_object.ComputePointPairPenetration()
            return len(collision_pairs) > 0
        return False

    def safe_path(self, q_start: tuple, q_end: tuple) -> list[tuple]:
        """Check for collisions along a straight-line path."""
        path = [q_start]
        max_steps = int(np.linalg.norm(np.array(q_end) - np.array(q_start)) / (np.pi / 180))
        max_steps = max(max_steps, 2)
        
        for step in range(1, max_steps + 1):
            alpha = step / max_steps
            q_interp = tuple(
                q_start[i] + alpha * (q_end[i] - q_start[i]) for i in range(len(q_start))
            )
            if self.collide(q_interp):
                return path
            path.append(q_interp)
        
        return path


class RRT_tools:
    """Tools for RRT algorithm."""
    def __init__(self, problem: DrawerProblem) -> None:
        self.rrt_tree = RRT(TreeNode(problem.start), problem.cspace)
        self.problem = problem

    def find_nearest_node_in_RRT_graph(self, q_sample: tuple) -> TreeNode:
        return self.rrt_tree.nearest(q_sample)

    def sample_node_in_configuration_space(self) -> tuple:
        return self.problem.cspace.sample()

    def calc_intermediate_qs_wo_collision(
        self, q_start: tuple, q_end: tuple
    ) -> list[tuple]:
        return self.problem.safe_path(q_start, q_end)

    def grow_rrt_tree(self, parent_node: TreeNode, q_sample: tuple) -> TreeNode:
        return self.rrt_tree.add_configuration(parent_node, q_sample)

    def node_reaches_goal(self, node: TreeNode, tol: float = 1e-2) -> bool:
        return self.problem.cspace.distance(node.value, self.problem.goal) <= tol

    def backup_path_from_node(self, node: TreeNode) -> list[tuple]:
        path = [node.value]
        while node.parent is not None:
            node = node.parent
            path.append(node.value)
        path.reverse()
        return path


class RRT_Connect_tools(RRT_tools):
    """Tools for RRT-Connect algorithm."""
    def create_new_tree(self, q_root: tuple[float]) -> RRT:
        return RRT(TreeNode(q_root), self.problem.cspace)

    def extend_once(
        self, tree: RRT, q_target: tuple[float], eps_connect: float = 1e-3
    ) -> tuple[Literal["Trapped", "Reached", "Advanced"], TreeNode]:
        q_near_node = tree.nearest(q_target)
        edge = self.problem.safe_path(q_near_node.value, q_target)
        if len(edge) <= 1:
            return "Trapped", q_near_node

        q_step = edge[1]
        new_node = tree.add_configuration(q_near_node, q_step)

        reached = q_step == q_target
        if not reached:
            if self.problem.cspace.distance(q_step, q_target) <= eps_connect:
                tail_edge = self.problem.safe_path(q_step, q_target)
                if len(tail_edge) > 1 and tail_edge[-1] == q_target:
                    new_node = tree.add_configuration(new_node, q_target)
                    reached = True

        return ("Reached" if reached else "Advanced"), new_node

    def connect_greedy(
        self, tree: RRT, q_target: tuple[float], eps_connect: float = 1e-3
    ) -> tuple[Literal["Trapped", "Reached", "Advanced"], TreeNode | None]:
        status, last = "Advanced", None
        while status == "Advanced":
            status, last = self.extend_once(tree, q_target, eps_connect)
            if status == "Trapped":
                return "Trapped", last
        return "Reached", last

    @staticmethod
    def concat_paths(path_a: list[tuple], path_b: list[tuple]) -> list[tuple]:
        if path_a and path_b and path_a[-1] == path_b[0]:
            return path_a + path_b[1:]
        return path_a + path_b


def rrt_connect_planning(
    problem: DrawerProblem, max_iterations: int = 1000, eps_connect: float = 1e-2
) -> tuple[list[tuple] | None, int]:
    """RRT-Connect in configuration space."""
    tools = RRT_Connect_tools(problem)

    T_start = tools.rrt_tree
    T_goal = tools.create_new_tree(problem.goal)

    active_is_start = True

    for it in range(1, max_iterations + 1):
        q_rand = tools.sample_node_in_configuration_space()

        T_active = T_start if active_is_start else T_goal
        T_other = T_goal if active_is_start else T_start

        status_a, node_a = tools.extend_once(T_active, q_rand, eps_connect)
        if status_a != "Trapped":
            q_new = node_a.value
            status_b, node_b = tools.connect_greedy(T_other, q_new, eps_connect)

            if status_b == "Reached" and node_b is not None:
                path_a = tools.backup_path_from_node(node_a)
                path_b = tools.backup_path_from_node(node_b)

                if not active_is_start:
                    path_a, path_b = path_b, path_a

                full_path = tools.concat_paths(path_a, list(reversed(path_b)))
                return full_path, it

        active_is_start = not active_is_start

    return None, max_iterations

In [ ]:
def solve_ik_for_grasp_pose(plant, iiwa_model, wsg_model, X_goal, q_guess=None):
    """Solve IK to find joint configuration for a desired gripper pose."""
    from pydrake.multibody import inverse_kinematics
    from pydrake.all import Solve, SolutionResult

    gripper_frame = plant.GetBodyByName("body", wsg_model).body_frame()
    world_frame = plant.world_frame()

    iiwa_start = plant.GetJointByName("iiwa_joint_1").position_start()

    # Try with decreasing tolerances
    tolerances = [
        (0.02, 0.174),   # 2cm position, ~10 degrees orientation
        (0.03, 0.262),   # 3cm position, ~15 degrees orientation
        (0.05, 0.349),   # 5cm position, ~20 degrees orientation
    ]

    for position_tolerance, orientation_tolerance in tolerances:
        ik = inverse_kinematics.InverseKinematics(plant)

        # Add constraints
        lower_bound = X_goal.translation() - np.array([position_tolerance] * 3)
        upper_bound = X_goal.translation() + np.array([position_tolerance] * 3)
        ik.AddPositionConstraint(
            gripper_frame, [0, 0, 0], world_frame,
            lower_bound, upper_bound
        )

        ik.AddOrientationConstraint(
            gripper_frame, RotationMatrix(),
            world_frame, X_goal.rotation(),
            orientation_tolerance
        )

        prog = ik.prog()
        q_vars = ik.q()

        # Set initial guess for IIWA joints
        if q_guess is not None:
            prog.SetInitialGuess(q_vars[iiwa_start:iiwa_start+7], q_guess)

        result = Solve(prog)

        if result.is_success():
            q_sol = result.GetSolution(q_vars)
            q_iiwa = q_sol[iiwa_start:iiwa_start+7]

            # Verify the solution is accurate by checking forward kinematics
            plant_context = plant.CreateDefaultContext()
            plant.SetPositions(plant_context, iiwa_model, q_iiwa)
            gripper_body = plant.GetBodyByName("body", wsg_model)
            X_achieved = plant.EvalBodyPoseInWorld(plant_context, gripper_body)

            pos_error = np.linalg.norm(X_achieved.translation() - X_goal.translation())

            print(f"  IK solved: pos_error={pos_error*1000:.2f}mm (tol={position_tolerance*1000:.1f}mm)")

            return q_iiwa

    print(f"  IK failed with all tolerances")
    return None

In [ ]:
def generate_trajectory(scenario_string, drawer_name="label53", num_approach_pts=50):
    try:
        # Initializing
        scenario = LoadScenario(data=scenario_string)
        builder = DiagramBuilder()
        station = MakeHardwareStation(scenario, meshcat=meshcat)
        builder.AddSystem(station)
        plant = station.GetSubsystemByName("plant")
        scene_graph = station.GetSubsystemByName("scene_graph")

        # Get initial poses of gripper and objects
        temp_context = station.CreateDefaultContext()
        temp_plant_context = plant.GetMyContextFromRoot(temp_context)
        drawer_model_instance = plant.GetModelInstanceByName("drawer")
        wsg_model = plant.GetModelInstanceByName("wsg")
        iiwa_model = plant.GetModelInstanceByName("iiwa")

        # Get current robot configuration
        q_start = plant.GetPositions(temp_plant_context, iiwa_model)

        handle_name = DRAWER_CONFIGS[drawer_name]["handle_label"]
        handle_body = plant.GetBodyByName(handle_name, drawer_model_instance)
        X_W_Handle = plant.EvalBodyPoseInWorld(temp_plant_context, handle_body)

        gripper_body = plant.GetBodyByName("body", wsg_model)

        # Compute desired task poses
        X_W_Grasp = grasp_pose(X_W_Handle)
        X_W_Pregrasp = approach_pose(X_W_Grasp)

        # Solve IK for pregrasp pose
        print("Solving IK for pregrasp pose...")
        q_pregrasp = solve_ik_for_grasp_pose(plant, iiwa_model, wsg_model, X_W_Pregrasp, q_guess=q_start)

        if q_pregrasp is None:
            print("ERROR: IK failed for pregrasp!")
            return None

        print(f"IK pregrasp succeeded")
        
        # Solve IK for grasp pose
        print("Solving IK for grasp pose...")
        q_grasp = solve_ik_for_grasp_pose(plant, iiwa_model, wsg_model, X_W_Grasp, q_guess=q_pregrasp)
        
        if q_grasp is None:
            print("ERROR: IK failed for grasp!")
            return None
        
        print(f"IK grasp succeeded")
        
        # Compute goal and post-goal poses
        X_W_Goal = goal_pose(X_W_Grasp, X_W_Handle)
        X_W_PostGoal = post_goal_pose(X_W_Goal, X_W_Handle)

        # Solve IK for goal and post-goal poses
        print("Solving IK for goal pose...")
        q_goal = solve_ik_for_grasp_pose(plant, iiwa_model, wsg_model, X_W_Goal, q_guess=q_grasp)

        if q_goal is None:
            print("ERROR: IK failed for goal!")
            return None

        print("Solving IK for post-goal pose...")
        q_postgoal = solve_ik_for_grasp_pose(plant, iiwa_model, wsg_model, X_W_PostGoal, q_guess=q_goal)

        if q_postgoal is None:
            print("ERROR: IK failed for post-goal!")
            return None

        print("Planning RRT path to pregrasp configuration...")

        # Plan RRT path
        problem_to_pregrasp = DrawerProblem(
            q_start=q_start,
            q_goal=q_pregrasp,
            plant=plant,
            scene_graph=scene_graph,
            station=station,
            station_context=temp_context,
            drawer_name=drawer_name
        )
        rrt_path, num_iters = rrt_connect_planning(problem_to_pregrasp, max_iterations=500, eps_connect=1e-2)

        if rrt_path is None:
            print("ERROR: RRT planning failed!")
            return None

        print(f"RRT found path in {num_iters} iterations with {len(rrt_path)} waypoints")

        # Phase definitions with pause times
        # RRT phase: move to pregrasp
        rrt_to_pregrasp_qs = rrt_path
        
        # Pause at pregrasp (1 second = 20 steps at 0.05s per step)
        num_pause_pregrasp_pts = 20
        pause_pregrasp_qs = [q_pregrasp] * num_pause_pregrasp_pts

        # Approach to grasp: slow and controlled
        num_approach_to_grasp_pts = num_approach_pts
        approach_to_grasp_qs = []
        for i in range(num_approach_to_grasp_pts):
            alpha = i / (num_approach_to_grasp_pts - 1)
            q_interp = (1 - alpha) * q_pregrasp + alpha * q_grasp
            approach_to_grasp_qs.append(q_interp)

        # Pause at grasp (2 seconds = 40 steps, let gripper close and grip)
        num_pause_grasp_pts = 40
        pause_grasp_qs = [q_grasp] * num_pause_grasp_pts

        # Grasp to goal phase (pull drawer) - slow and smooth
        num_grasp_to_goal_pts = 60
        grasp_to_goal_qs = []
        for i in range(num_grasp_to_goal_pts):
            alpha = i / (num_grasp_to_goal_pts - 1)
            q_interp = (1 - alpha) * q_grasp + alpha * q_goal
            grasp_to_goal_qs.append(q_interp)

        # Pause at goal (1 second = 20 steps, hold position)
        num_pause_goal_pts = 20
        pause_goal_qs = [q_goal] * num_pause_goal_pts

        # Goal to post-goal phase - slow and smooth
        num_goal_to_postgoal_pts = 60
        goal_to_postgoal_qs = []
        for i in range(num_goal_to_postgoal_pts):
            alpha = i / (num_goal_to_postgoal_pts - 1)
            q_interp = (1 - alpha) * q_goal + alpha * q_postgoal
            goal_to_postgoal_qs.append(q_interp)

        # Pause at post-goal (1 second = 20 steps)
        num_pause_postgoal_pts = 20
        pause_postgoal_qs = [q_postgoal] * num_pause_postgoal_pts

        # Combine all phases
        full_rrt_path = (rrt_to_pregrasp_qs + pause_pregrasp_qs +
                        approach_to_grasp_qs + pause_grasp_qs +
                        grasp_to_goal_qs + pause_goal_qs + 
                        goal_to_postgoal_qs + pause_postgoal_qs)

        # Visualize the trajectory using the same method as rrt_planning notebook
        print("\nVisualizing trajectory in Meshcat and capturing images...")
        scenario = LoadScenario(data=scenario_string)
        builder_vis = DiagramBuilder()
        station_vis = MakeHardwareStation(scenario, meshcat=meshcat)
        builder_vis.AddSystem(station_vis)
        plant_vis = station_vis.GetSubsystemByName("plant")

        # Create trajectory from full path
        times = [0.05 * i for i in range(len(full_rrt_path))]
        Q = np.column_stack(full_rrt_path)
        traj = PiecewisePolynomial.FirstOrderHold(times, Q)

        # Create gripper command sequence
        wsg_commands = []
        
        # RRT phase: gripper closed
        for i in range(len(rrt_to_pregrasp_qs)):
            wsg_commands.append(np.array([0.0]))

        # Pause at pregrasp: gripper closing
        for i in range(len(pause_pregrasp_qs)):
            wsg_commands.append(np.array([0.0]))

        # Approach to grasp: gripper opening
        for i in range(len(approach_to_grasp_qs)):
            wsg_commands.append(np.array([1.0]))

        # Pause at grasp: gripper closing to grasp - hold tight
        for i in range(len(pause_grasp_qs)):
            wsg_commands.append(np.array([0.0]))

        # Grasp to goal: gripper closed and held tight
        for i in range(len(grasp_to_goal_qs)):
            wsg_commands.append(np.array([0.0]))

        # Pause at goal: gripper closed
        for i in range(len(pause_goal_qs)):
            wsg_commands.append(np.array([0.0]))

        # Goal to post-goal: gripper open
        for i in range(len(goal_to_postgoal_qs)):
            wsg_commands.append(np.array([1.0]))

        # Pause at post-goal: gripper open
        for i in range(len(pause_postgoal_qs)):
            wsg_commands.append(np.array([1.0]))
        
        wsg_cmd_array = np.column_stack(wsg_commands)
        wsg_traj = PiecewisePolynomial.FirstOrderHold(times, wsg_cmd_array)

        # Setup trajectory sources
        iiwa_src = builder_vis.AddSystem(TrajectorySource(traj))
        wsg_src = builder_vis.AddSystem(TrajectorySource(wsg_traj))

        builder_vis.Connect(iiwa_src.get_output_port(), station_vis.GetInputPort("iiwa.position"))
        builder_vis.Connect(wsg_src.get_output_port(), station_vis.GetInputPort("wsg.position"))

        diagram = builder_vis.Build()

        # Run simulation and record
        simulator = Simulator(diagram)
        context = simulator.get_mutable_context()
        diagram.ForcedPublish(context)
        
        end_time = traj.end_time()
        print(f"Running simulation for {end_time:.2f} seconds...")
        
        # Storage for camera frames
        camera0_images = []
        camera1_images = []
        camera2_images = []
        
        try:
            meshcat.StartRecording()
            
            # Simulate and capture images at regular intervals
            dt = 0.05  # Match the trajectory time step
            current_time = 0.0
            
            while current_time <= end_time:
                simulator.AdvanceTo(current_time)
                
                # Get the correct context for station_vis from the diagram context
                station_vis_context = station_vis.GetMyContextFromRoot(context)
                
                # Get camera outputs
                camera0_output_port = station_vis.GetOutputPort("camera0.rgb_image")
                camera1_output_port = station_vis.GetOutputPort("camera1.rgb_image")
                camera2_output_port = station_vis.GetOutputPort("camera2.rgb_image")
                
                # Evaluate the cameras at the correct context
                camera0_image = camera0_output_port.Eval(station_vis_context)
                camera1_image = camera1_output_port.Eval(station_vis_context)
                camera2_image = camera2_output_port.Eval(station_vis_context)
                
                # Convert to numpy arrays if needed
                if hasattr(camera0_image, 'data'):
                    camera0_images.append(np.array(camera0_image.data, dtype=np.uint8))
                else:
                    camera0_images.append(np.zeros((480, 640, 3), dtype=np.uint8))
                    
                if hasattr(camera1_image, 'data'):
                    camera1_images.append(np.array(camera1_image.data, dtype=np.uint8))
                else:
                    camera1_images.append(np.zeros((480, 640, 3), dtype=np.uint8))
                    
                if hasattr(camera2_image, 'data'):
                    camera2_images.append(np.array(camera2_image.data, dtype=np.uint8))
                else:
                    camera2_images.append(np.zeros((480, 640, 3), dtype=np.uint8))
                
                current_time += dt
            
            meshcat.StopRecording()
            meshcat.PublishRecording()
            print(f"Visualization complete! Captured {len(camera0_images)} frames")
            
        except Exception as e:
            print(f"ERROR: Simulation crashed during visualization: {e}")
            print(f"Simulation time reached: {simulator.get_context().get_time():.2f}s / {end_time:.2f}s")
            import traceback
            traceback.print_exc()
            return None

        # Return trajectory data with full path waypoints and actual camera images
        qpos_data = [np.array(q) for q in full_rrt_path]
        qvel_data = [np.zeros(7) for _ in full_rrt_path]
        
        # Action includes joint velocities (7) + gripper command (1)
        action_data = []
        for i, wsg_cmd in enumerate(wsg_commands):
            action = np.concatenate([np.zeros(7), wsg_cmd])
            action_data.append(action)

        return {
            "qpos": qpos_data,
            "qvel": qvel_data,
            "action": action_data,
            "images": {
                "camera0": camera0_images if camera0_images else [np.zeros((480, 640, 3), dtype=np.uint8) for _ in full_rrt_path],
                "camera1": camera1_images if camera1_images else [np.zeros((480, 640, 3), dtype=np.uint8) for _ in full_rrt_path],
                "camera2": camera2_images if camera2_images else [np.zeros((480, 640, 3), dtype=np.uint8) for _ in full_rrt_path]
            },
            "timestamp": [float(i) for i in range(len(full_rrt_path))]
        }

    except Exception as e:
        print(f"ERROR in generate_trajectory: {e}")
        import traceback
        traceback.print_exc()
        return None

In [ ]:
def save_trajectory_data(data: dict, episode_idx: int = 0, drawer_name="label53"):
    # ---------- convert buffers to numpy arrays ----------
    qpos_arr = np.stack(data['qpos'], axis=0).astype(np.float32)      # (T, nq)
    qvel_arr = np.stack(data['qvel'], axis=0).astype(np.float32)      # (T, nq)
    action_arr = np.stack(data['action'], axis=0).astype(np.float32)  # (T, act_dim)

    images0_arr = np.stack(data['images']['camera0'], axis=0).astype(np.uint8)  # (T, H, W, 3)
    images1_arr = np.stack(data['images']['camera1'], axis=0).astype(np.uint8)  # (T, H, W, 3)
    images2_arr = np.stack(data['images']['camera2'], axis=0).astype(np.uint8)  # (T, H, W, 3)

    timestamps_arr = np.array(data['timestamp'], dtype=np.float64)

    # ---------- write ACT-style HDF5 episode ----------
    h5_path = os.path.join("teleop_data/sim_open_drawer", f"episode_{episode_idx}_{drawer_name}.hdf5")
    print("Saving episode to ", h5_path)

    with h5py.File(h5_path, "w") as root:
        # mark this as simulation data (ACT uses this flag)
        root.attrs["sim"] = True

        # actions: shape (T, act_dim)
        root.create_dataset("action", data=action_arr, compression="gzip")

    # observations group
        obs_grp = root.create_group("observations")
        obs_grp.create_dataset("qpos", data=qpos_arr, compression="gzip")
        obs_grp.create_dataset("qvel", data=qvel_arr, compression="gzip")

        # optional: store timestamps (not strictly required by ACT but often useful)
        obs_grp.create_dataset("time", data=timestamps_arr)

        # images subgroup, one dataset per camera
        img_grp = obs_grp.create_group("images")
        img_grp.create_dataset("camera0", data=images0_arr, compression="gzip")
        img_grp.create_dataset("camera1", data=images1_arr, compression="gzip")
        img_grp.create_dataset("camera2", data=images2_arr, compression="gzip")

    print(f"Wrote ACT-style episode to {h5_path}")

In [ ]:
num_episodes_per_drawer = 5
max_retries = 3  # Retry each failed episode up to 3 times

# train_drawers = ["label31", "label53", "label62"]
train_drawers = ["label53"]
eval_drawers = [""]

for i in range(num_episodes_per_drawer):
    for drawer in train_drawers:
        print(f"\n{'='*60}")
        print(f"Generating episode {i} for drawer {drawer}...")
        print(f"{'='*60}")
        
        retry_count = 0
        trajectory_data = None
        
        while retry_count < max_retries and trajectory_data is None:
            if retry_count > 0:
                print(f"\nRetry attempt {retry_count}/{max_retries}...")
            
            try:
                scenario_string = generate_scenario_string(drawer_name=drawer)
                trajectory_data = generate_trajectory(scenario_string, drawer_name=drawer)
            except Exception as e:
                print(f"EXCEPTION during episode generation: {e}")
                import traceback
                traceback.print_exc()
                trajectory_data = None
            
            retry_count += 1
        
        # Skip saving if trajectory generation failed after all retries
        if trajectory_data is None:
            print(f"\nEpisode {i} for {drawer} FAILED after {max_retries} retries - SKIPPING\n")
            continue
        
        try:
            save_trajectory_data(trajectory_data, episode_idx=i, drawer_name=drawer)
            print(f"Episode {i} for {drawer} SUCCESSFULLY COMPLETED.\n")
        except Exception as e:
            print(f"ERROR saving trajectory data: {e}")
            import traceback
            traceback.print_exc()
        
        time.sleep(3)  # Longer wait to let animation finish before next episode


Generating episode 0 for drawer label31...
0.4120684413035144


material [ 'None_NONE.006' ] not found in .mtl



Solving IK for pregrasp pose...
  IK solved: pos_error=34.36mm (tol=20.0mm)
IK pregrasp succeeded
Solving IK for grasp pose...
  IK solved: pos_error=34.64mm (tol=20.0mm)
IK grasp succeeded
Solving IK for goal pose...
  IK solved: pos_error=34.16mm (tol=20.0mm)
Solving IK for post-goal pose...
  IK solved: pos_error=33.78mm (tol=20.0mm)
Planning RRT path to pregrasp configuration...
RRT found path in 1 iterations with 99 waypoints

Visualizing trajectory in Meshcat...


material [ 'None_NONE.006' ] not found in .mtl



Visualization complete!
Saving episode to  teleop_data/sim_open_drawer/episode_0_label31.hdf5
Wrote ACT-style episode to teleop_data/sim_open_drawer/episode_0_label31.hdf5
Episode 0 for label31 SUCCESSFULLY COMPLETED.


Generating episode 0 for drawer label53...
0.3067708125268762


material [ 'None_NONE.006' ] not found in .mtl



Solving IK for pregrasp pose...
  IK solved: pos_error=26.48mm (tol=20.0mm)
IK pregrasp succeeded
Solving IK for grasp pose...
  IK solved: pos_error=34.64mm (tol=20.0mm)
IK grasp succeeded
Solving IK for goal pose...
  IK solved: pos_error=32.94mm (tol=20.0mm)
Solving IK for post-goal pose...
  IK solved: pos_error=34.64mm (tol=20.0mm)
Planning RRT path to pregrasp configuration...
RRT found path in 1 iterations with 155 waypoints

Visualizing trajectory in Meshcat...


material [ 'None_NONE.006' ] not found in .mtl



Visualization complete!


### Watching the demonstration back

In [ ]:
import imageio; 
from IPython.display import Video; 

%matplotlib inline

In [ ]:
import imageio; 
from IPython.display import Video; 

%matplotlib inline

num_episodes = 3
drawers = ["label31", "label53", "label62"]

for drawer in drawers:
    for i in range(num_episodes):
        file_path = f"teleop_data/sim_open_drawer/episode_{i}_{drawer}.hdf5"
        
        try:
            f = h5py.File(file_path, "r")
            data = f['observations']['images']['camera0']
            
            imageio.mimwrite(f'episode_playback_{i}_{drawer}.mp4', data, fps=30)
            print(f"Created video: episode_playback_{i}_{drawer}.mp4")
            
            f.close()
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error processing {file_path}: {e}")